In [1]:
# modules
from tqdm import  tqdm
from glob import glob
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import basename as bn, join, split as sp

import librosa
import parselmouth
from parselmouth.praat import call
from scipy.io.wavfile import write

import praat_formants_python as pfp


# DATASET Path and Constants

In [2]:
# Read paths
ROOT_TIMIT_DATA_PATH = "/home/jeevan/datasets/TIMIT Acoustic-Phonetic Continuous Speech Corpus (LDC93S1)/TIMIT"

# Write paths
ALL_EXP_FOLDER = "./exports/"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(ALL_EXP_FOLDER) #make export folder

# Vowel info Export CSV filename
ALL_TIMIT_VOWELS_IMP_FILENAME = "a_all-timit_vowels.csv"
ALL_TIMIT_VOWELS_IMP_FILEPATH = join(ALL_EXP_FOLDER, ALL_TIMIT_VOWELS_IMP_FILENAME)

# Vowel subset Export CSV filename
SUBSET_TIMIT_VOWELS_IMP_FILENAME = "b_subset-timit_vowels_vowlimLIM.csv"
SUBSET_TIMIT_VOWELS_IMP_FILEPATH = join(ALL_EXP_FOLDER, SUBSET_TIMIT_VOWELS_IMP_FILENAME)

# Vowel subset Export CSV filename
TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILENAME = "c_timit-vowels_formant_estimation_vowlimLIM.csv"
TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH = join(ALL_EXP_FOLDER, TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILENAME)


# TMP Audio Export folder
TEMP_AUDIO_EXP_FOLDER = "./audio_exports"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(TEMP_AUDIO_EXP_FOLDER) #make export folder

# TIMIT SAMPLING RATE
TIMIT_AUDIO_FS = 16000

### Import SUBSET TIMIT Vowel Info dataframe

In [3]:
VOWEL_LIMIT = 100
SUBSET_TIMIT_VOWELS_DF = pd.read_csv(SUBSET_TIMIT_VOWELS_IMP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}"))
# SUBSET_TIMIT_VOWELS_DF.set_index("index", inplace=True)
ALL_TIMIT_VOWEL_LIST = pd.unique(SUBSET_TIMIT_VOWELS_DF["vowel_name"])
print(SUBSET_TIMIT_VOWELS_DF["vowel_name"].value_counts())
SUBSET_TIMIT_VOWELS_DF

iy      200
ae      200
uh      200
ey      200
ah      200
aw      200
ux      200
ax      200
ay      200
oy      200
eh      200
ix      200
ow      200
axr     200
ao      200
ih      200
uw      200
aa      200
er      200
ax-h     21
Name: vowel_name, dtype: int64


,index,audio_filepath,wav_file,person_id,sex,start_sample,end_sample,duration_sample,start_second,end_second,duration_second,vowel_name
0,0,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA2.WAV,MWGR0,M,8482,9539,1057,0.530125,0.596187,0.066062,iy
1,1,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX394.WAV,MJES0,M,23410,24600,1190,1.463125,1.537500,0.074375,iy
2,2,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI2222.WAV,MSTK0,M,56557,57800,1243,3.534813,3.612500,0.077688,iy
3,3,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI1340.WAV,MJDM0,M,43947,45891,1944,2.746688,2.868187,0.121500,iy
4,4,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI1239.WAV,MLBC0,M,39979,41357,1378,2.498687,2.584812,0.086125,iy
...,...,...,...,...,...,...,...,...,...,...,...,...
3816,3816,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI1448.WAV,MRJM3,M,48800,50168,1368,3.050000,3.135500,0.085500,ax-h
3817,3817,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI1923.WAV,MAPV0,M,39477,40600,1123,2.467312,2.537500,0.070187,ax-h
3818,3818,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX92.WAV,MSFV0,M,40647,43446,2799,2.540437,2.715375,0.174937,ax-h
3819,3819,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI2241.WAV,FLJG0,F,30033,31206,1173,1.877063,1.950375,0.073313,ax-h


## FUNCTION: Measure Pitch of audio chunk | PARSEL MOUTH

In [4]:
def measure_pitch(audio_path: str) -> float:
    f0min, f0max = [75, 600]
    
    sound = parselmouth.Sound(audio_path) # read the sound
    pitch = call(sound, "To Pitch", 0, f0min, f0max) #create a praat pitch object
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz") # get mean pitch
    return mean_pitch

## FUNCTION: Measure formants of audio chunk | PARSEL MOUTH

In [5]:
def measure_formants_psm(audio_path: str, vowel_name: str, start_sec: float, end_sec: float):
    f0min, f0max = [75, 600]
    sound = parselmouth.Sound(audio_path) # read the sound
    # pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pitch = call(sound, "To Pitch", 0.0001, f0min, f0max)
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz") # get mean pitch
    
    audio_chunk, fs = librosa.load(audio_path, sr=None, offset=start_sec, duration=(end_sec - start_sec))
    tmp_audio_file = os.path.join(TEMP_AUDIO_EXP_FOLDER, f"{vowel_name}.wav")
    write(tmp_audio_file, fs, audio_chunk)
    sound_frm = parselmouth.Sound(tmp_audio_file)
    # sound_frm = sound_frm.extract_part(rom_time=start_sec, to_time=end_sec, window_shape=0, relative_width=1, preserve_times=False) # read the sound chunk
    pointProcess = call(sound_frm, "To PointProcess (periodic, cc)", f0min, f0max)
    formants = call(sound_frm, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t  = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    f1_mean = np.mean(f1_list)
    f2_mean = np.mean(f2_list)
    f3_mean = np.mean(f3_list)
    f4_mean = np.mean(f4_list)
    
    # calculate median formants across pulses, this is what is used in all subsequent calcualtions
    # you can use mean if you want, just edit the code in the boxes below to replace median with mean
    f1_median = np.median(f1_list)
    f2_median = np.median(f2_list)
    f3_median = np.median(f3_list)
    f4_median = np.median(f4_list)
    
    return {
        "pitch_mean_praat_base": mean_pitch,

        "F1_mean_praat_base":f1_mean,
        "F2_mean_praat_base":f2_mean,
        "F3_mean_praat_base":f3_mean,
        "F4_mean_praat_base":f4_mean,

        "F1_median_praat_base": f1_median,
        "F2_median_praat_base": f2_median,
        "F3_median_praat_base": f3_median,
        "F4_median_praat_base": f4_median,
    }

## FUNCTION: Measure formants of audio chunk | PRAAT FORMANTS

In [6]:
def measure_formants_pfp(audio_path, start_sec, end_sec):
    formants = pfp.formants_at_interval(
        audio_path, start_sec, end_sec, maxformant=5500, winlen=0.025, preemph=50
    )

    pitch_mean = measure_pitch(audio_path)
    pitch_mean = np.round(pitch_mean, 2)
    
    formants_mean = formants.mean(axis=0)
    formants_mean = list(formants_mean)[1:]  # skip time
    formants_mean = np.round(formants_mean, 2)  # round

    formants_median = np.median(formants, axis=0)
    formants_median = list(formants_median)[1:]  # skip time
    formants_median = np.round(formants_median, 2) # round


    return {
        "pitch_mean_praat_base": pitch_mean,

        "F1_mean_praat_base": formants_mean[0],
        "F2_mean_praat_base": formants_mean[1],
        "F3_mean_praat_base": formants_mean[2],

        "F1_median_praat_base": formants_median[0],
        "F2_median_praat_base": formants_median[1],
        "F3_median_praat_base": formants_median[2],
    }


### FUNCTION TESTS

In [15]:
audf, start, end, v = SUBSET_TIMIT_VOWELS_DF.loc[np.random.randint(0, len(SUBSET_TIMIT_VOWELS_DF)), ["audio_filepath", "start_second", "end_second", "vowel_name"]]
print(audf, start, end)
print(measure_formants_pfp(audf, start, end))

/home/jeevan/datasets/TIMIT Acoustic-Phonetic Continuous Speech Corpus (LDC93S1)/TIMIT/TEST/DR4/FNMR0/SX49.WAV 2.0365 2.1488125
{'pitch_mean_praat_base': 181.68, 'F1_mean_praat_base': 679.05, 'F2_mean_praat_base': 1739.83, 'F3_mean_praat_base': 2440.7, 'F1_median_praat_base': 691.71, 'F2_median_praat_base': 1737.26, 'F3_median_praat_base': 2454.01}


## FUNCTION: TIMIT Vowel PITCH, FORMANT Estimation

In [16]:
def estimate_vowel_formants(vowel_info):
    audio_file = vowel_info["audio_filepath"]
    start_sec = vowel_info["start_second"]
    end_sec = vowel_info["end_second"]
    vowel_name = vowel_info["vowel_name"]

    formant_estimates = measure_formants_pfp(audio_file, start_sec, end_sec)
    # formant_estimates = measure_formants_psm(audio_file, vowel_name, start_sec, end_sec)

    new_vowel_info = dict(vowel_info) | formant_estimates

    return new_vowel_info

vinfo = SUBSET_TIMIT_VOWELS_DF.loc[np.random.randint(0, len(SUBSET_TIMIT_VOWELS_DF))]
estimate_vowel_formants(vinfo)

{'index': 3565,
 'audio_filepath': '/home/jeevan/datasets/TIMIT Acoustic-Phonetic Continuous Speech Corpus (LDC93S1)/TIMIT/TRAIN/DR6/FSDJ0/SX35.WAV',
 'wav_file': 'SX35.WAV',
 'person_id': 'FSDJ0',
 'sex': 'F',
 'start_sample': 12685,
 'end_sample': 13911,
 'duration_sample': 1226,
 'start_second': 0.7928125,
 'end_second': 0.8694375,
 'duration_second': 0.076625,
 'vowel_name': 'ey',
 'pitch_mean_praat_base': 180.41,
 'F1_mean_praat_base': 535.08,
 'F2_mean_praat_base': 1762.37,
 'F3_mean_praat_base': 2462.27,
 'F1_median_praat_base': 551.18,
 'F2_median_praat_base': 1755.7,
 'F3_median_praat_base': 2438.51}

### Create and export Vowel Formant Estimation Result dataframe

In [17]:
VOWELS_FORMANT_DF = pd.DataFrame([estimate_vowel_formants(v_i) for i, v_i in tqdm(SUBSET_TIMIT_VOWELS_DF[0:].iterrows())])

columns = ['index',  'person_id', 'sex', 'duration_second', 'vowel_name', 
           'pitch_mean_praat_base', 
           'F1_mean_praat_base', 'F2_mean_praat_base', 'F3_mean_praat_base', 
           'F1_median_praat_base', 'F2_median_praat_base', 'F3_median_praat_base']

csv_path = TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}")
if not os.path.exists(csv_path):
    VOWELS_FORMANT_DF.to_csv(csv_path, columns=columns, index=False)

VOWELS_FORMANT_DF = VOWELS_FORMANT_DF.loc[:, columns]
VOWELS_FORMANT_DF

3821it [07:15,  8.77it/s]


,index,person_id,sex,duration_second,vowel_name,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,F2_median_praat_base,F3_median_praat_base
0,0,MWGR0,M,0.066062,iy,118.56,495.03,2131.18,2950.80,495.03,2124.55,2957.00
1,1,MJES0,M,0.074375,iy,112.66,316.93,2098.74,2844.58,325.92,2098.60,2876.64
2,2,MSTK0,M,0.077688,iy,150.65,426.11,1931.06,2447.48,420.31,2046.93,2372.78
3,3,MJDM0,M,0.121500,iy,94.78,467.55,1824.06,2588.08,450.22,1824.37,2533.09
4,4,MLBC0,M,0.086125,iy,107.83,460.67,1942.34,2558.01,461.12,2026.29,2558.57
...,...,...,...,...,...,...,...,...,...,...,...,...
3816,3816,MRJM3,M,0.085500,ax-h,112.14,743.40,1492.04,2672.56,671.51,1352.75,2550.95
3817,3817,MAPV0,M,0.070187,ax-h,133.61,646.22,1847.19,2720.36,613.69,1837.41,2696.50
3818,3818,MSFV0,M,0.174937,ax-h,99.52,765.19,1876.51,3195.29,600.50,1647.24,2806.36
3819,3819,FLJG0,F,0.073313,ax-h,195.82,852.51,1813.99,2963.22,861.90,1809.55,2976.29


### Create and export Vowel Formant Estimation Result dataframe: JSON

In [19]:
json_fp = TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}").replace(".csv", ".json")

if not os.path.exists(json_fp):
    VOWELS_FORMANT_DF.to_json(
        json_fp, index=False, orient="table"
    )

print(json_fp)

./exports/c_timit-vowels_formant_estimation_vowlim100.json
